In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from suite2p.extraction import dcnv

In [ ]:
# compute deconvolution

tau = 1.0 # timescale of indicator
fs = 30.0 # sampling rate in Hz
neucoeff = 0.7 # neuropil coefficient
# for computing and subtracting baseline
baseline = 'maximin' # take the running max of the running min after smoothing with␣

# gaussian
sig_baseline = 10.0 # in bins, standard deviation of gaussian with which to smooth
win_baseline = 60.0 # in seconds, window in which to compute max/min filters
prctile_baseline = 8
batch_size = 200
ops = {'tau': tau, 'fs': fs, 'neucoeff': neucoeff, 'baseline': baseline, 'sig_baseline': sig_baseline, 
       'win_baseline': win_baseline, 'prctile_baseline': prctile_baseline, 'batch_size': batch_size}

# load traces and subtract neuropil
F = np.load(os.path.join('./files', 'suite2p', 'plane0', 'F.npy'))
Fneu = np.load(os.path.join('./files', 'suite2p', 'plane0', 'Fneu.npy'))#np.load('Fneu.npy')
Fc = F - ops['neucoeff'] * Fneu

# baseline operation
Fc = dcnv.preprocess(
    F=Fc,
    baseline=ops['baseline'],
    win_baseline=ops['win_baseline'],
    sig_baseline=ops['sig_baseline'],
    fs=ops['fs'],
    prctile_baseline=ops['prctile_baseline']
)

# get spikes
spks = dcnv.oasis(F=Fc, batch_size=ops['batch_size'], tau=ops['tau'], fs=ops['fs'])

In [ ]:
plt.plot(spks.flatten())

In [ ]:
plt.figure(figsize=(16, 4))
plt.imshow((spks*255).astype(np.uint8))